In [1]:
from agents import *
#from scipy.spatial import distance

class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            print("Dog: Ate food at {}.".format(self.location))
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            print("Dog: Drank water at {}.".format(self.location))
            return True
        return False

    
def program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

class Park(Environment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        print(things)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog pick eat the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will or it with when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


class Food(Thing):
    pass

class Water(Thing):
    pass


In [2]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 0)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(10)

[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>]
[<BlindDog>, <Food>]
Dog: Ate food at 5.
[<BlindDog>]
[<BlindDog>]
[<BlindDog>, <Water>]
Dog: Drank water at 7.


In [3]:

class Hero(Agent):
    location = [1,1]
    points = 100
    direction = "right"
    
    def advance(self):
        if(self.direction == "right"):
            self.location[1]+=1
        elif(self.direction == "left"):
            self.location[1]-=1
        elif(self.direction== "up"):
            self.location[0] -=1
        elif(self.direction== "down"):
            self.location[0] += 1
    def turn(self):
        if(self.direction == "right"):
            self.direction = "down"
        elif(self.direction == "left"):
            self.direction = "up"
        elif(self.direction== "up"):
            self.direction ="right"
        elif(self.direction== "down"):
            self.direction = "left"
    def stay(self):
        print("Stay")
    def grab(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Gold):
            print("I found a coint at:  {}.!".format(self.location))
            self.points += 5
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            print("Dog: Drank water at {}.".format(self.location))
            self.points -= 50
            return True
        return False

def onedge(ap):
    #if (p.location[1] == 4 and p.direction == "right") or (p.location[1] == 0 and p.direction == "left") or (p.location[0] == 4 and p.direction == "down") or (p.location[0] == 0 and p.direction == "up"):
    #    return True
    if (ap[0] >= 5 or ap[1] >= 5 or ap[1] <= -1 or ap[0] <= -1):
        return True
    else:
        return False

def manhattan(loc1,loc2):
    return abs(loc1[0]-loc2[0])+abs(loc1[1]-loc2[1])

def getgoal(perps,p):
    closestdistance = 99
    for k in perps:
        if isinstance(k[0], Gold):
            if closestdistance == 99 or (manhattan(p[1],k[1]) <= closestdistance):
                closestdistance = manhattan(p[1],k[1])
                goal = k[1]
            #print("current closest distance = "+str(goal))
    return goal

def getgoalanddist(perps,p):
    closestdistance = 99
    for k in perps:
        if isinstance(k[0], Gold):
            if closestdistance == 99 or (manhattan(p[1],k[1]) <= closestdistance):
                closestdistance = manhattan(p[1],k[1])
                goal = k[1]
            #print("current closest distance = "+str(goal))
    return goal,closestdistance

def getadvanceposition(p):
    if p[0].direction=="right":
        return [p[1][0],p[1][1]+1]
    elif p[0].direction == "left":
        return [p[1][0],p[1][1]-1]
    elif p[0].direction == "up":
        return [p[1][0]-1,p[1][1]]
    elif p[0].direction == "down":
        return [p[1][0]+1,p[1][1]]
    
    
    
def istheregold(perps):
    for p in perps:
        if isinstance(p[0],Gold):
            return True
    return False
    
    
def program2(percepts):
    '''Returns an action based on it's percepts'''
    if istheregold(percepts) == False:
        return 'stay'
    else:
        for p in percepts:
            if isinstance(p[0], Hero):
                goal = getgoalanddist(percepts,p) #Closest gold piece
                ap = getadvanceposition(p) #Next position in current trajectory
                print("My goal is: "+str(goal[0])+" and my next move is: "+str(ap))
                if onedge(ap):
                    return 'turn'
                else:
                    if manhattan(goal[0],ap) < goal[1]:
                        print("I have to move!")
                        return 'advance'
                    elif (manhattan(goal[0],p[1]) == 0):
                        return 'grab'
                    else: 
                        return 'turn'
    return 'advance'

In [4]:

class Gold(Thing):
    pass

class Trap(Thing):
    pass


class Grid(Environment):
    def percept(self, agent):
        things = []
        '''prints & return a list of things that are in our agent's location'''
        for x in range(0,5):
            for y in range(0,5):
                if(len(self.list_things_at([x,y]))!= 0):
                    for thing in self.list_things_at([x,y]):                        
                        things.append([thing,[x,y]])        
        print(things)
        return things
    
    def execute_action(self, agent, action):
        chargold = ""
        chartrap = ""
        charhero = "R"
        '''changes the state of the environment based on what the agent does.'''
        if action == "advance":
            agent.advance()
        elif action == "turn":
            agent.turn()
        elif action == "grab":
            items = self.list_things_at(agent.location, tclass=Gold)
            if len(items) != 0:
                if agent.grab(items[0]): #Have the dog pick eat the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.            
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog pick eat the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "stay":
            agent.stay()
        for x in range(0,5):
            for y in range(0,5):
                if(agent.location == [x,y]):
                    if(agent.direction == "right"):
                        charhero = 'R'
                    elif(agent.direction == "left"):
                        charhero = 'L'
                    elif(agent.direction == "up"):
                        charhero = 'U'
                    elif(agent.direction == "down"):
                        charhero = "D"
                else:
                    charhero = '-'
                    
                thingsgold = self.list_things_at([x,y], tclass=Gold)
                thingstrap = self.list_things_at([x,y],tclass=Trap)
                if len(thingsgold) != 0 or len(thingstrap) != 0:
                    if(len(thingsgold) == 0):
                        chargold = "-"
                    else:
                        chargold = str(len(thingsgold))
                    if(len(thingstrap) == 0):
                        chartrap = "-"
                    else: 
                        chartrap = str(len(thingstrap))
                    print("("+charhero+","+chargold+","+chartrap+") ", end = '')
                else:
                    print("("+charhero+",-,-) ", end = '')
            print("\n")
        print(str(agent.points))
            
            
    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "Don't add the same thing twice"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)
    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will or it with when there is no more food or water'''
        #no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents


In [5]:
grid = Grid()
hero = Hero(program2)
goldpiece = Gold()
goldpiece2 = Gold()
goldpiece3 = Gold()

trap = Trap()
grid.add_thing(hero, [0,0])
grid.add_thing(goldpiece, [2,2])
grid.add_thing(trap, [3,3])
grid.add_thing(goldpiece2, [2,2])
grid.add_thing(goldpiece3, [1,1])
grid.run(20)

[[<Hero>, [0, 0]], [<Gold>, [1, 1]], [<Gold>, [2, 2]], [<Gold>, [2, 2]], [<Trap>, [3, 3]]]
My goal is: [1, 1] and my next move is: [0, 1]
I have to move!
(-,-,-) (R,-,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (-,1,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (-,-,-) (-,2,-) (-,-,-) (-,-,-) 

(-,-,-) (-,-,-) (-,-,-) (-,-,1) (-,-,-) 

(-,-,-) (-,-,-) (-,-,-) (-,-,-) (-,-,-) 

100
[[<Hero>, [0, 1]], [<Gold>, [1, 1]], [<Gold>, [2, 2]], [<Gold>, [2, 2]], [<Trap>, [3, 3]]]
My goal is: [1, 1] and my next move is: [0, 2]
(-,-,-) (D,-,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (-,1,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (-,-,-) (-,2,-) (-,-,-) (-,-,-) 

(-,-,-) (-,-,-) (-,-,-) (-,-,1) (-,-,-) 

(-,-,-) (-,-,-) (-,-,-) (-,-,-) (-,-,-) 

100
[[<Hero>, [0, 1]], [<Gold>, [1, 1]], [<Gold>, [2, 2]], [<Gold>, [2, 2]], [<Trap>, [3, 3]]]
My goal is: [1, 1] and my next move is: [1, 1]
I have to move!
(-,-,-) (-,-,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (D,1,-) (-,-,-) (-,-,-) (-,-,-) 

(-,-,-) (-,-,-) (-,2,-) (-,-,-) (-,-,-) 

